# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import re
import json
!pip install pdfplumber
import pdfplumber
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Get Portuguese stop words
portuguese_stop_words = set(stopwords.words('portuguese'))
english_stop_words = set(stopwords.words('english'))

In [ ]:
# As funções abaixo foram adaptadas de: https://github.com/jsvine/pdfplumber/issues/356#issuecomment-1471361607

# Retorna se um objeto não está contido em outro
# Por exemplo: se um texto está contido em uma tabela ou figura
def not_within_bboxes(obj,bboxes):

    def obj_in_bbox(_bbox):
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)

    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)

def curves_to_edges(cs):
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

# Extrai o texto de um arquivo PDF que não está dentro de tabelas ou figuras
def raw_text_extract(pdf_file, include_tables=False, include_images=False, show_page_number=False):
    page_data = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            #page = pdf.pages[0]

            if show_page_number:
                print(f"Pagina: {page.page_number}")

            #print(page.extract_text())
            bboxes = []
            # identificando as tabelas
            if not include_tables:
                bboxes = [
                    table.bbox
                    for table in page.find_tables(
                    table_settings={
                        "vertical_strategy": "lines",
                        "horizontal_strategy": "lines",
                        "explicit_vertical_lines": curves_to_edges(page.curves) + page.edges,
                        "explicit_horizontal_lines": curves_to_edges(page.curves) + page.edges,
                    }
                    )
                ]
            #print(bboxes)

            # identificando as imagens
            if not include_images:
                for image in page.images:
                    image_bbox = (image['x0'], image['top'], image['x1'], image['bottom'])
                    bboxes.append(image_bbox)
                    #print("img: ",image_bbox)

            # Filtrando os textos que estão fora das caixas das tabelas e das figuras
            page = page.filter(lambda obj: not_within_bboxes(obj, bboxes))

            # a arquivo bbdc-2015-3T-Transcrição da Teleconferência 3T15.pdf gera caracteres duplicados. Esse método resolve o problema.
            # ref: https://github.com/jsvine/pdfplumber/issues/71
            text = page.dedupe_chars().extract_text()


            ##### removendo cabeçalho: bbdc, bbas, prbc -> "Transc... ano": Pode ter ou não "Transcrição da"; Pode ter 3 ou 4 linhas

            # Transcrição da Teleconferência
            # Resultados do 4T09
            # Banco do Brasil (BBAS3 BZ) <- esta linha pode nao aparecer no BBDC
            # 26 de fevereiro de 2010

            # prbc-2013-2T13
            # Teleconferência do Paraná Banco
            # Resultados do 2° trimestre de 2013
            # 14 de junho de 2013 – 11h00 (horário de Brasília)
            text = re.sub(r"(Transcrição da )?Teleconferência(.*)?((?:\n|\r\n?)(.*))?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)[0-9]{4}( – [0-9]{2}h(.*))?", '', text).strip()

            ##### Removendo cabeçalhos do ITUB
            # Itaú Unibanco
            # Resultados do Terceiro trimestre de 2018
            # 30 de outubro de 2018
            text = re.sub(r"Itaú Unibanco(.*)?((?:\n|\r\n?)(.*))?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)[0-9]{4}", '', text).strip()

            ##### Removendo cabeçalhos do BBAS
            # - Palavra "#Pública" (3T19 a 3T22) e #interna (1T21 a 1T22)
            text = re.sub(f"#Pública", '', text).strip()

            # - Palavra "#interna" (1T21 a 1T22)
            text = re.sub(f"#interna", '', text).strip()


            # bbas-2006-4T06-Transcrição
            # Local Conference Call
            # Banco do Brasil Nac. – (29314)
            # Resultados do Exercício de 2006
            # 28 de Fevereiro de 2007 – 11:00h - horário local
            text = re.sub(r"Local Conference Call(?:\n|\r\n?)(.*)?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)", '', text).strip()

            # bbas-2009-4T09-Transcrição.pdf
            # O padrão é este. As linhas iniciais são removidas na regra anterior, mas fica a linha da data, que é removida aqui
            # Transcrição da Teleconferência
            # Resultados do 4T09
            # Banco do Brasil (BBAS3 BZ)
            # 26 de fevereiro de 2010
            text = re.sub(r"^[0-9]{2}(.*)[0-9]{4}", '', text).strip()

            # Relação com Investidores
            # Transcrição 1T21
            # BANCO DO BRASIL
            # TELECONFERÊNCIA
            # DE RESULTADOS
            # 1T2021
            text = re.sub(r"Relação com Investidores(?:\n|\r\n?)(.*)?((?:\n|\r\n?)(.*))?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)?(?:\n|\r\n?)(.*)", '', text).strip()

            # bbas-2020-3T20-Transcrição Teleconferência 3T20.pdf
            # BANCO DO BRASIL
            # TELECONFERÊNCIA
            # DE RESULTADOS
            # 3T2020
            # 06/11/2020
            text = re.sub(r"BANCO DO BRASIL(?:\n|\r\n?)(.*)?((?:\n|\r\n?)(.*))?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)", '', text).strip()


            # removendo cabeçalho: abcb -> "Banco ABC Brasil | Relações com Investidores Transcrição da"
            text = re.sub(r"Banco ABC Brasil \| (.*)((?:\n|\r\n?))", '', text).strip()


            # removendo o número da página que veio junto do texto
            text = re.sub(f"\\n{page.page_number}$", '', text)

            text = re.sub(f'- {page.page_number} -', '', text)


            # removendo o número da página que veio junto do texto: bbdc -> "(cid:1) <pagina>"
            text = re.sub(f"\(cid:1\) {page.page_number}$", '', text).strip()

            # removendo o número da página que veio junto do texto: prbc -> "Página <pagina>"
            text = re.sub(f"Página {page.page_number}$", '', text).strip()

            # removendo o número da página que veio junto do texto: itub -> "1/12"
            text = re.sub(f'{page.page_number}/{len(pdf.pages)}$', '', text).strip()

            # removendo o número da página que veio junto do texto: itub -> "Teleconferência 4T21 2"
            text = re.sub(r'Teleconferência \dT\d{2} \d+$', '', text).strip()

            # removendo o número da página que veio junto do texto:
            # No abcp-2009-3T09, todas as páginas estão com número "9" e
            # no abcp-2009-3T09, todas as páginas estão com número 7
            head, tail = os.path.split(pdf_file)
            if (tail.startswith('abcb-2009')):
                if (tail.startswith('abcb-2009-3T09')):
                    text = re.sub(r'((?:\n|\r\n?))9$', '', text).strip()
                elif (tail.startswith('abcb-2009-4T09')):
                    text = re.sub(r'((?:\n|\r\n?))7$', '', text).strip()

            page_data.append(text)

    return page_data

In [ ]:
def clean(text):

    text = text.replace("\n", ' ')
    text = text.replace("”", '')
    text = text.replace("“", '')
    text = text.replace("\"", '')
    text = text.replace("", '')

    #lista com bolinha
    text = re.sub(r'(\s)?(;\s)?(•)', "; ", text.strip())
    text = re.sub(r'(: ;)', ": ", text.strip())
    text = re.sub(r'(\.;)', ". ", text.strip())

    # nu-2021-4T21-Script 4T21.pdf
    text = re.sub(r'(; -)', ". ", text.strip())
    text = re.sub(r'(\. -)', ". ", text.strip())
    text = re.sub(r'^(-)', "", text.strip())
    text = re.sub(r'(: \d.)', ': .', text.strip())
    text = re.sub(r'(; e (\d+\.)?)', '.', text.strip())
    text = re.sub(r'(: ●)', '.', text.strip())
    text = re.sub(r'(; ●)', '.', text.strip())
    text = re.sub(r'(●)', '', text.strip())

    text = re.sub("_______________________________________________________________", "", text)

    text = re.sub(r"Sra\.", "Senhora ", text, flags=re.IGNORECASE)
    text = re.sub(r"Sr\.", "Senhor ", text, flags=re.IGNORECASE)
    text = re.sub(r"Srs\.", "Senhores ", text, flags=re.IGNORECASE)
    text = re.sub(r'b\.p\.\s([A-Z])', 'bp. \\1', text).strip()
    text = re.sub('b.p.', 'bp', text).strip()
    text = re.sub('p.p.', 'pp', text).strip()
    text = re.sub('help!', 'help', text).strip() # bmgb -> tirar a exclamação para evitar quebra de sentenças

    text = re.sub('\s+', ' ', text).strip() # deixar por ultimo, pois as substituicoes anteriores podem inserir multiplos espaços

    return text

In [ ]:
def get_sentences(text):
    sentences = sent_tokenize(text, language='portuguese')
    sentences = [s for s in sentences if len(s.strip()) > 2]
    return sentences

def get_tokens(text):
    tokens = word_tokenize(text, language='portuguese')

    return tokens

def get_words(text):
    tokens = get_tokens(text)

    words = [w for w in tokens if w not in string.punctuation]

    return words

In [ ]:
def generate_stats(text):
    stats = {}
    tokens = get_tokens(text)
    num_tokens = len(tokens)

    stats["tokens"] = num_tokens

    return stats

# Split Text

In [ ]:
def split_text(text):
  # Define patterns to identify the start of the Q&A section
  qna_start_patterns = [
    "Estamos abertos para perguntas que vocês possam ter",
    'Era basicamente isso que a gente tinha para falar, então a gente pode ir agora para as perguntas',
    "Encerrando a apresentação, eu gostaria agora de abrir para Perguntas e Respostas"
  ]

  # Join all patterns into a single regex pattern
  pattern = r"|".join(qna_start_patterns)

  match = re.search(pattern, text, re.IGNORECASE)
  if match:
      split_index = match.start()
      presentation = text[:split_index].strip()
      qna = text[split_index:].strip()
      return presentation, qna
  else:
      return text, None

In [ ]:
transcript_folder = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Transcripts/'
transcript_text_folder = "/content/drive/MyDrive/Portfolio Projects/Mestrado/transcricoes_processadas/"

global_stats = pd.DataFrame()

#Recupera os arquivos que estão na pasta
files = os.listdir(transcript_folder)
files = files[0:2]

num_files = len(files)
for i, file in enumerate(files):
    path_transcription = transcript_folder + file

    file_name_parts = file.split("-")
    ticker          = file_name_parts[0].strip()
    trimestre       = file_name_parts[2].strip()

    path_folder_ticker = os.path.join(transcript_text_folder, ticker)

    print(path_folder_ticker, ticker, trimestre)
    print('path_transcription', path_transcription)
    if not os.path.exists(path_folder_ticker):
        os.mkdir(path_folder_ticker)

    print('is_file',os.path.isfile(path_transcription))
    if os.path.isfile(path_transcription):
      print(f"Processando arquivo {i+1} de {num_files}")

      print("*** Extraindo texto do PDF: "+ file)
      path_transcription = transcript_folder + file
      if ticker in ['bbas', 'bbdc']:
          page_data = raw_text_extract(path_transcription)
      else:
          page_data = raw_text_extract(path_transcription, include_tables=True, include_images=False)

      #print(page_data)

      print("*** Limpando o texto")
      text = " ".join(page_data)

In [ ]:
#!pip install openai
#!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet
import openai
api_key = "api_key"
openai.api_key = api_key

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
# Define the function to evaluate the text with a prompt
def evaluate_text_with_prompt(text, prompt, model="gpt-4-turbo"):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Você deve concentrar os outputs para serem estritamente o que pedi, pois vou usá-los para códigos em pipeline"},
            {"role": "user", "content": prompt},
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message.content


In [ ]:
transcript_folder = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Transcripts/'
transcript_text_folder = "/content/drive/MyDrive/Portfolio Projects/Mestrado/Generated Text 2/"

#Recupera os arquivos que estão na pasta
files = os.listdir(transcript_text_folder)
files = sorted(files)

cutoff = {}

for file_path in files:
  path_transcription = transcript_text_folder + file_path

  file_name_parts = file_path.split("-")
  ticker          = file_name_parts[0].strip()
  ano             = file_name_parts[1].strip()
  trimestre       = file_name_parts[2].strip()[0]

  print(ticker,ano,trimestre)

  with open(path_transcription, 'r') as file:
    text_content = file.read()

    # Example prompt
    prompt = f"""
      Você poderia indicar o trecho do texto que posso usar como cutoff para separar o texto da apresentação e o das perguntas e repostas (Q&A)?

      Por favor não inclua nada do enunciado na resposta, apenas o trecho do texto até a primeira pontuação do Q&A.
    """

    response = evaluate_text_with_prompt(text_content, prompt) #evaluate_text_with_prompt(before_text, prompt)

    cutoff[f'{ticker}-{ano}-{trimestre}'] = response

df_cutoff = pd.DataFrame.from_dict(cutoff, orient='index', columns=['cutoff'])
df_cutoff.head()

In [ ]:
file_path = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Temp'
df_cutoff.to_csv(f'{file_path}/df_cutoff_raw.csv', index=True)

In [ ]:
i = 0
df_cutoff['revision'] = 0
for index, row in df_cutoff.iterrows():
  i = i+1
  print(i+1)

      # Example prompt
  prompt = f"""
    Você poderia me indicar se o trecho indica que ainda haverá uma fala antes do início da sessão de perguntas e resposta?

    Gostaria que a resposta viesse como uma opção binária: (Sim) ou (Não).
  """

  print(text_content, prompt)

  response = evaluate_text_with_prompt(row['cutoff'], prompt) #evaluate_text_with_prompt(before_text, prompt)
  print(ticker, ano, trimestre, response)

  df_cutoff.loc[index,'revision'] = response

df_cutoff.head()

In [ ]:
#Search File name saved
#file_path = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Temp'
#df_cutoff.to_csv(f'{file_path}/df_cutoff_raw.csv', index=True)

In [ ]:
transcript_folder = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Transcripts/'
transcript_text_folder = "/content/drive/MyDrive/Portfolio Projects/Mestrado/Generated Text 2/"

#Recupera os arquivos que estão na pasta
files = os.listdir(transcript_text_folder)
files = sorted(files)

revision_list = list(df_cutoff[df_cutoff['revision'] == 'Sim']['doc'])

revised_cutoff = {}

#banco do brasil: files[59:125]
#for file_path in files:
for file_path in files:
  print('file_path',file_path)
  path_transcription = transcript_text_folder + file_path

  file_name_parts = file_path.split("-")
  print(file_name_parts)
  ticker          = file_name_parts[0].strip()
  ano             = file_name_parts[1].strip()
  trimestre       = file_name_parts[2].strip()[0]

  if f'{ticker}-{ano}-{trimestre}' in revision_list:
    print(ticker,ano,trimestre, 'YES')

    with open(path_transcription, 'r') as file:
      text_content = file.read()

    prompt = f"""
      Você poderia indicar o trecho do texto que posso usar como cutoff para separar o texto da apresentação e o das perguntas e repostas (Q&A)?

      Certifique-se de que não haja nenhuma seção depois do trecho e antes das perguntas e respostas. Mesmo que seja apenas um comentário ou última explicação.

      Se o trecho anunciar que haverá alguma fala antes das perguntas e respostas ele não está correto.

      Por favor não inclua nada do enunciado na resposta, apenas o trecho do texto até a primeira pontuação do Q&A.
    """

    print(text_content, prompt)

    response = evaluate_text_with_prompt(text_content, prompt) #evaluate_text_with_prompt(before_text, prompt)
    print(ticker, ano, trimestre, response)

    revised_cutoff[f'{ticker}-{ano}-{trimestre}'] = response

df_revised_cutoff = pd.DataFrame.from_dict(revised_cutoff, orient='index', columns=['cutoff'])
df_revised_cutoff.head()

In [ ]:
df_cutoff_merge = df_cutoff.merge(df_revised_cutoff, on='doc', how='left', suffixes=('_original', '_revised'))
df_cutoff_merge['cutoff_revised'] = df_cutoff_merge.apply(lambda x: x['cutoff_revised'] if x['revision'] == 'Sim' else x['cutoff_original'],axis=1)
df_cutoff_merge.head()

In [ ]:
file_path = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Temp'
df_cutoff_merge.to_csv(f'{file_path}/df_cutoff_merge.csv', index=True)

## Manual Adjustments

In [ ]:
file_path = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Temp/df_cutoff_merge - antigo.csv'
df_cutoff_merge = pd.read_csv(file_path)
df_cutoff_merge.drop(columns=df_cutoff_merge.columns[0], axis=1)
df_cutoff_merge.head(15)

In [ ]:
import os
import re
import pandas as pd
import concurrent.futures
from functools import partial

# Example folder paths
transcript_folder = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Transcripts/'
transcript_text_folder = "/content/drive/MyDrive/Portfolio Projects/Mestrado/transcricoes_processadas_BERTopic/"

# Suppose these are your files
files = os.listdir(transcript_folder)
files = files#[:10]  # Just first 10 as an example

def process_single_file(file_name, df_cutoff_merge):
    """
    Worker function that processes a single file.
    Returns a dictionary with doc name, 'presentation', 'qna' segments, and their lengths.
    """
    path_transcription = os.path.join(transcript_folder, file_name)

    # Split file name to extract relevant parts
    file_name_parts = file_name.split("-")
    ticker    = file_name_parts[0].strip()
    ano       = file_name_parts[1].strip()
    trimestre = file_name_parts[2].strip()

    # Decide on how to call raw_text_extract based on ticker
    if ticker in ['bbas', 'bbdc']:
        page_data = raw_text_extract(path_transcription)
    else:
        page_data = raw_text_extract(path_transcription, include_tables=True, include_images=False)

    # JOIN the list of lines/pages into a single string
    text = " ".join(page_data)

    # Build the doc_name for lookup in df_cutoff_merge
    doc_name = f'{ticker}-{ano}-{trimestre[0]}'

    # Retrieve the cutoff pattern for this doc
    # IMPORTANT: Make sure doc_name actually matches something in df_cutoff_merge
    # e.g. 'bbas-:2023-1'
    cutoff_rows = df_cutoff_merge[df_cutoff_merge['doc'] == doc_name]
    if len(cutoff_rows) == 0:
        # If no matching row, skip or handle logic
        # For now, treat entire text as presentation
        return {
            'doc': doc_name,
            'presentation': text,
            'qna': None,
            'presentation_len': len(text),
            'qna_len': 0
        }

    cutoff_str = cutoff_rows['joint_cutoff'].values[0]
    print(doc_name, cutoff_str)

    # Escape special regex chars to avoid unbalanced parentheses or similar
    pattern = re.escape(cutoff_str)

    # Search for the pattern in the text
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        split_index = match.start()
        presentation = text[:split_index].strip()
        qna = text[split_index:].strip()
    else:
        # If no match, entire text is 'presentation'
        presentation = text
        qna = None

    # Create a row-like dictionary
    new_row = {
        'doc': doc_name,
        'cutoff': cutoff_str,
        'presentation': presentation,
        'qna': qna,
        'presentation_len': len(presentation),
        'qna_len': len(qna) if qna else 0
    }
    return new_row

# Prepare an empty DataFrame to gather results
df_divided = pd.DataFrame(columns=['doc','presentation','qna','presentation_len','qna_len'])

num_files = len(files)
texts_as_strings = []

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    # Use functools.partial to pass df_cutoff_merge
    process_func = partial(process_single_file, df_cutoff_merge=df_cutoff_merge)

    # Executor map returns an iterator of dictionaries (one per file)
    results = executor.map(process_func, files)

    for i, row_dict in enumerate(results, start=1):
        print(f"Processed file {i} of {num_files}")

        # row_dict is the dictionary returned by process_single_file
        # We can build texts_as_strings if needed
        presentation_text = row_dict['presentation'] if row_dict['presentation'] else ""
        qna_text = row_dict['qna'] if row_dict['qna'] else ""
        combined_text = presentation_text + " " + qna_text
        texts_as_strings.append(combined_text)

        # Append row to df_divided
        df_divided = pd.concat([df_divided, pd.DataFrame([row_dict])], ignore_index=True)

print("All files processed.")
print(df_divided.head())

In [ ]:
df_divided['cutoff_revised'] = df_divided['cutoff']
#abcb
df_divided.iloc[226,6] = 'É basicamente isso, estamos abertos para perguntas que vocês possam ter.'
#bbas
df_divided.iloc[22,6]  = 'Operadora: Obrigada. Com licença, iniciaremos agora a sessão de perguntas e respostas.'
df_divided.iloc[283,6] = 'OPERADORA – Senhoras e senhores, iniciaremos agora a sessão de perguntas e respostas. Para fazer uma pergunta, por favor, digitem *1.'
df_divided.iloc[253,6] = 'OPERADORA – Senhoras e senhores, iniciaremos agora a Sessão de Perguntas e Respostas. Para fazer uma pergunta, por favor, digitem *1.'
df_divided.iloc[347,6] = 'OPERADORA- Senhoras e senhores, iniciaremos agora a sessão de perguntas e respostas.'
df_divided.iloc[143,6] = 'OPERADORA – Obrigada. Iniciaremos agora a sessão de perguntas e respostas'
df_divided.iloc[215,6] = 'Agora me junto ao Cassiano e ao Firetti para a sessão de perguntas e respostas.'
#bmgb
df_divided.iloc[191,6] = 'gostaria de abrir para sessões de perguntas e respostas'
df_divided.iloc[295,6] = 'Danilo, abro agora a sessão de perguntas.'
df_divided.iloc[92,6]  = 'isso, encerro também a apresentação e abro para a sessão de perguntas.'
df_divided.iloc[131,6]  = 'Com isso, encerramos a nossa apresentação e vamos passar para perguntas e respostas'
df_divided.iloc[383,6]  = 'Com isso, encerramos aqui a nossa apresentação e vamos abrir agora para perguntas'
df_divided.iloc[219,6]  = 'Dessa forma, eu encerro aqui a apresentação de resultados do Banco Bmg e'
df_divided.iloc[9,6]  = 'Encerro aqui a apresentação. Iniciaremos a nossa sessão de perguntas e respostas'
df_divided.iloc[296,6]  = 'E assim, eu encerro aqui a apresentação de resultados, para iniciarmos a sessão, agora'
df_divided.iloc[352,6]  = 'Vamos passar para o Q&A agora.'
df_divided.iloc[89,6]  = 'E assim, eu encerro a apresentação e gostaria de abrir para sessões de perguntas'
#bpan
df_divided.iloc[274,6]  = 'E aí, com isso, a gente encerra os slides e pode começar o nosso Q&A com os analistas e investidores'
#banrisul
df_divided.iloc[90,6]  = 'Então, acho que era isso que eu queria comentar, e estamos à disposição para perguntas dos senhores'
df_divided.iloc[6,6]  = 'Eram essas as observações que eu tinha a fazer, e agora aguardamos os questionamentos para ver as questões que nós podemos clarear. Obrigado'
df_divided.iloc[113,6]  = 'Vou encerrar por enquanto, e o grupo aqui ficará à disposição para eventuais perguntas de quem está nos prestigiando aqui hoje. Muito obrigado.'
df_divided.iloc[236,6]  = 'eu só queria fazer isso com essa consideração final e já podemos passar, então, para as perguntas e respostas.'
df_divided.iloc[66,6]  = 'Com isso, a apresentação em si se encerra e devolvemos a palavra aos senhores e às senhoras para as perguntas que desejarem. Ficamos à vontade.'
df_divided.iloc[19,6]  = 'Com isso, eu encerro a discussão dos principais números, e nós nos colocamos à disposição dos analistas para os questionamentos que houver. Obrigado.'
df_divided.iloc[204,6]  = 'Com isso, vamos passar para a parte dos questionamentos do mercado, para'
df_divided.iloc[199,6]  = 'E agradeço a todos pela participação nessa áudioconferência. Obrigado'
df_divided.iloc[323,6]  = 'Com isso aqui eu encerro a apresentação e devolvo para a continuidade da nossa reunião.'
#itub
df_divided.iloc[188,6]  = 'Com isso encerramos a apresentação e estamos aqui abertos todos nós para as perguntas que vocês quiserem fazer.'
df_divided.iloc[85,6]  = 'Com isso eu encerro a apresentação. Nós estamos aqui todos abertos às perguntas que vocês queiram fazer.'
df_divided.iloc[310,6]  = 'Essas são as telas que a gente estaria apresentando, agora estamos abertos às perguntas de vocês.'
df_divided.iloc[162,6]  = 'Muito bem, tendo dito tudo isto, eu estou abrindo para perguntas e obrigado aí pela atenção de vocês.'
df_divided.iloc[355,6]  = 'Com isso eu encerro aqui a apresentação e estaremos a partir de agora, o Marcelo Kopel e eu, à disposição para responder eventuais perguntas. Obrigado.'
df_divided.iloc[291,6]  = 'Bem, com estas projeções, eu concluo a apresentação e nós podemos passar para as perguntas e respostas.'
df_divided.iloc[4,6]  = 'Com isso, nós concluímos esta apresentação e estamos abertos agora a qualquer questão que vocês possam ter. Muito obrigado.'
df_divided.iloc[194,6]  = 'Com isto, eu concluo a apresentação, estou aberto para qualquer dúvida que vocês possam ter. Muito obrigado.'
df_divided.iloc[250,6]  = 'Com isso, nós concluímos esta apresentação e estamos abertos agora a qualquer questão que vocês possam ter. Muito obrigado.'
df_divided.iloc[232,6]  = 'E, com isso, eu encerro aqui a apresentação e abro para perguntas.'
df_divided.iloc[322,6]  = 'chamar o Renato de volta para me ajudar com a sessão de perguntas e respostas.'
df_divided.iloc[377,6]  = 'Agora vou para o nosso Perguntas e Respostas, vou me juntar ao Renato. Nos vemos'
#prbc
df_divided.iloc[234,6]  = 'Não existe'
df_divided.iloc[177,6]  = 'Operador: Nossa primeira pergunta vem do senhor Francisco do banco Safra.'
#santander
df_divided.iloc[251,6]  = 'Então, muito obrigado pela sua atenção. Quando vocês quiserem, podemos'
df_divided.iloc[38,6]  = 'Bom dia. Será que vocês poderiam nos dar um pouco mais de detalhe da natureza'
df_divided.iloc[329,6]  = 'Com isso, eu concluo, e creio que agora teremos nossa sessão de perguntas e respostas. Muito obrigado.'
df_divided.iloc[218,6]  = 'Obrigado por sua atenção e agora estamos disponíveis para responder às suas perguntas.'
df_divided.iloc[245,6]  = 'Eu gostaria de agradecer a todos pela atenção. Agora podemos iniciar a sessão de perguntas e respostas.'
df_divided.iloc[17,6]  = 'Gostaria de agradecer a todos pela atenção e ficamos à disposição responder as suas perguntas'
df_divided.iloc[195,6]  = 'Então, era isso, em poucas palavras, que eu queria compartilhar com vocês, e acho que agora podemos abrir a sessão para perguntas e respostas.'
df_divided.iloc[21,6]  = 'Com isso, faço uma pausa e teremos, acredito, meia hora para perguntas e respostas'
#abcb
df_divided.iloc[102,6]  = 'Esses eram os principais fatos que a gente gostaria de apresentar, e nos colocamos à disposição para qualquer dúvida ou qualquer pergunta'
df_divided.iloc[197,6]  = 'a gente tinha para apresentar, a gente abre para qualquer dúvida, pra qualquer pergunta, ahn, que, que seja necessária. Obrigado.'
df_divided.iloc[87,6]  = 'Isso é basicamente o que a gente apostou, então a partir de agora a gente abre para responder às perguntas que vocês'
df_divided.iloc[109,6]  = 'Oi pessoal, é o Thiago Baptista. Eu tenho 2 perguntas. A primeira em relação as margens'
df_divided.iloc[357,6]  = 'Estes são os dados que gostaríamos de apresentar. Nos colocamos agora à disposição para responder'
df_divided.iloc[171,6]  = 'Para participar é só levantar a mão clicando no ícone na parte inferior da tela'
df_divided.iloc[64,6]  = 'transmissão. Agora, como mencionado, abriremos a seção de perguntas e respostas.'
#bbas
df_divided.iloc[137,6]  = 'Essas eram as considerações sobre detalhamento dos resultados. Gostaríamos agora de abrir a teleconferência para a sessão de perguntas e respostas'
df_divided.iloc[220,6]  = 'Agora podemos abrir a sessão de perguntas e respostas, e ficamos à disposição'
df_divided.iloc[269,6]  = 'Essas eram praticamente as informações que nós gostaríamos de compartilhar com vocês. Podemos agora abri para a sessão de perguntas e respostas. Muito obrigada.'
df_divided.iloc[373,6]  = 'Nossa primeira pergunta vem do senhor Eduardo Rosman, do Banco'
df_divided.iloc[255,6]  = 'Agradeço agora a presença de todos e podemos seguir para a sessão de perguntas e respostas'
df_divided.iloc[261,6]  = 'E agora agradeço a participação de todos e podemos iniciar a sessão de perguntas e respostas'
df_divided.iloc[115,6]  = 'Assim eu finalizo aqui a apresentação dos nossos números e gostaria de abrir, então para o nosso Q&A. Agradeço aí o tempo e a atenção'
#bbdc
df_divided.iloc[185,6]  = 'Muito obrigado pela participação de todos que estão conectados em nossa teleconferência, e a partir de agora estamos à disposição'
df_divided.iloc[70,6]  = 'Obrigada. Iniciaremos agora a sessão de perguntas e respostas. Para fazer uma pergunta, por favor, digitem “*1”, e se quiser retirar a pergunta da lista'
df_divided.iloc[208,6]  = 'Muito obrigado pela atenção das senhoras e dos senhores e passamos agora para a seção de perguntas e respostas. Muito obrigado'
df_divided.iloc[218,6]  = 'Obrigado pela atenção de todos até o momento e passamos agora para a seção de perguntas e respostas'
df_divided.iloc[135,6]  ='Finalizo por aqui essa primeira parte da reunião e vou me juntar ao Firetti no outro estúdio para seguirmos para a sessão de perguntas e respostas'
#bmgb
df_divided.iloc[110,6]  = 'Com isso eu finalizo a nossa apresentação, nossos slides, e abrimos agora para perguntas de vocês.'
df_divided.iloc[287,6]  = 'Com isso, encerro aqui a apresentação. Agora vamos abrir para as perguntas de vocês.'
#brsr
df_divided.iloc[68,6]  = 'Eu gostaria de encerrar por enquanto, e colocar então à disposição para eventuais perguntas que os analistas tenham a nos fazer. Muito obrigado até o momento.'
df_divided.iloc[210,6]  = 'São dois comentários importantes que eu gostaria de fazer antes de encaminhar as perguntas e respostas. Nós estamos então à disposição às perguntas dos analistas'
df_divided.iloc[154,6]  = 'Essas foram algumas considerações básicas que gostaríamos de fazer, e estamos à disposição, então, para eventuais perguntas. Obrigado.'
df_divided.iloc[242,6]  = 'Com isso concluímos a apresentação dos resultados desse 1S. Quero agradecer mais uma vez o interesse e a atenção de todos, e dizer que a partir'
df_divided.iloc[228,6]  = 'nossa área de RI em contato permanente para poder fornecer dados também por escrito, via email, etc. Então, aguardamos as questões dos senhores.'
df_divided.iloc[158,6]  = 'aos senhores e senhoras, para que possamos atendê-los nas demandas que houver. Vamos para a sessão de perguntas e respostas, então. Obrigado.'
df_divided.iloc[100,6]  = 'Então, basicamente é isso. Agradeço a todos a presença e aguardo questionamentos. Muito obrigado.'
df_divided.iloc[231,6]  = 'Iniciaremos nossa sessão de perguntas e respostas, que contará com a participação do Sr.Nathan Meneguzzi'
#itub
df_divided.iloc[151,6]  = 'Acho que é isso. Podemos abrir aqui para perguntas. Então, vamos a elas.'
df_divided.iloc[336,6]  = 'E, com isso, eu termino a apresentação de resultados e abro para perguntas e respostas'
df_divided.iloc[326,6]  = 'Está bom? Muito obrigado. Vou subir agora para o estúdio e vou encontrar com o Renato para que a gente possa continuar o nosso bate pa'
#nu
df_divided.iloc[302,6]  = "Não existe"
df_divided.iloc[124,6]  = 'Operadora: Iniciaremos agora a sessão de perguntas e respostas para investidores e analistas'
#prbc
df_divided.iloc[63,6]  = "Não Existe"
df_divided.iloc[298,6]  = "Não Existe"
df_divided.iloc[35,6]  = "Não Existe"
df_divided.iloc[174,6]  = "Não Existe"
df_divided.iloc[328,6]  = "Não Existe"
df_divided.iloc[198,6]  = "Não Existe"
#sanb
df_divided.iloc[55,6]  = 'Sem mais, eu passo a palavra ao moderador.'
df_divided.iloc[257,6]  = 'Muito obrigado pela sua participação, e agora abro para a sessão de perguntas e respostas.'
df_divided.iloc[108,6]  = 'Muito obrigado e estamos disponíveis para responder perguntas agora.'
df_divided.iloc[211,6]  = 'Com isso, eu concluo a minha apresentação. Acho que o André e eu podemos passar para a sessão de perguntas e respostas'

In [ ]:
import os
import re
import pandas as pd
import concurrent.futures
from functools import partial

# Example folder paths
transcript_folder = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Transcripts/'
transcript_text_folder = "/content/drive/MyDrive/Portfolio Projects/Mestrado/transcricoes_processadas_BERTopic/"

# Suppose these are your files
files = os.listdir(transcript_folder)
files = files#[:10]  # Just first 10 as an example

def process_single_file(file_name, df_cutoff_merge):
    """
    Worker function that processes a single file.
    Returns a dictionary with doc name, 'presentation', 'qna' segments, and their lengths.
    """
    path_transcription = os.path.join(transcript_folder, file_name)

    # Split file name to extract relevant parts
    file_name_parts = file_name.split("-")
    ticker    = file_name_parts[0].strip()
    ano       = file_name_parts[1].strip()
    trimestre = file_name_parts[2].strip()

    # Decide on how to call raw_text_extract based on ticker
    if ticker in ['bbas', 'bbdc']:
        page_data = raw_text_extract(path_transcription)
    else:
        page_data = raw_text_extract(path_transcription, include_tables=True, include_images=False)

    # JOIN the list of lines/pages into a single string
    text = " ".join(page_data)

    # Build the doc_name for lookup in df_cutoff_merge
    doc_name = f'{ticker}-{ano}-{trimestre[0]}'

    # Retrieve the cutoff pattern for this doc
    # IMPORTANT: Make sure doc_name actually matches something in df_cutoff_merge
    # e.g. 'bbas-:2023-1'
    cutoff_rows = df_divided[df_divided['doc'] == doc_name]
    if len(cutoff_rows) == 0:
        # If no matching row, skip or handle logic
        # For now, treat entire text as presentation
        return {
            'doc': doc_name,
            'presentation': text,
            'qna': None,
            'presentation_len': len(text),
            'qna_len': 0
        }

    cutoff_str = cutoff_rows['cutoff_revised'].values[0]
    print(doc_name, cutoff_str)

    # Escape special regex chars to avoid unbalanced parentheses or similar
    pattern = re.escape(cutoff_str)

    # Search for the pattern in the text
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        split_index = match.start()
        presentation = text[:split_index].strip()
        qna = text[split_index:].strip()
    else:
        # If no match, entire text is 'presentation'
        presentation = text
        qna = None

    # Create a row-like dictionary
    new_row = {
        'doc': doc_name,
        'cutoff': cutoff_str,
        'presentation': presentation,
        'qna': qna,
        'presentation_len': len(presentation),
        'qna_len': len(qna) if qna else 0
    }
    return new_row

# Prepare an empty DataFrame to gather results
df_divided_revised = pd.DataFrame(columns=['doc','presentation','qna','presentation_len','qna_len'])

num_files = len(files)
texts_as_strings = []

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    # Use functools.partial to pass df_cutoff_merge
    process_func = partial(process_single_file, df_cutoff_merge=df_cutoff_merge)

    # Executor map returns an iterator of dictionaries (one per file)
    results = executor.map(process_func, files)

    for i, row_dict in enumerate(results, start=1):
        print(f"Processed file {i} of {num_files}")

        # row_dict is the dictionary returned by process_single_file
        # We can build texts_as_strings if needed
        presentation_text = row_dict['presentation'] if row_dict['presentation'] else ""
        qna_text = row_dict['qna'] if row_dict['qna'] else ""
        combined_text = presentation_text + " " + qna_text
        texts_as_strings.append(combined_text)

        # Append row to df_divided
        df_divided_revised = pd.concat([df_divided_revised, pd.DataFrame([row_dict])], ignore_index=True)

print("All files processed.")
print(df_divided_revised.head())

In [ ]:
general_folder = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Divided_text/'
if not os.path.exists(general_folder):
    os.mkdir(general_folder)

transcript_presentation_folder = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Divided_text/presentation/'
transcript_qna_folder          = '/content/drive/MyDrive/Portfolio Projects/Mestrado/Divided_text/qna'

if not os.path.exists(transcript_presentation_folder):
    os.mkdir(transcript_presentation_folder)
if not os.path.exists(transcript_qna_folder):
    os.mkdir(transcript_qna_folder)

count = 0
num_files = len(df_divided_revised)
for index, row in df_divided_revised.iterrows():
  count += 1
  file_name = f"{row['doc']}.txt"
  print(f'Processing file {count} of {num_files}')

  file_path_presentation = os.path.join(transcript_presentation_folder, file_name)
  with open(file_path_presentation, 'w') as file:
    file.write(row['presentation'])

  if row['qna'] is not None:
    file_path_qna = os.path.join(transcript_qna_folder, file_name)
    with open(file_path_qna, 'w') as file:
      file.write(row['qna'])
  else:
    continue

print('All files processed')

# EDA